In [1]:
import sqlite3
import pandas as pd

In [2]:
pd.options.display.max_columns = None
#pd.options.display.max_rows = None

In [3]:
conn = sqlite3.connect('parch-and-posey.db')

In [4]:
cursor = conn.cursor()
cursor.execute('''
select * from sqlite_master where type = "table";
''')
columns = [col[0] for col in cursor.description]
data = cursor.fetchall()
cursor.close()

In [5]:
pd.DataFrame(data, columns=columns)

,type,name,tbl_name,rootpage,sql
0,table,web_events,web_events,2,"CREATE TABLE web_events (\tid integer,\taccoun..."
1,table,sales_reps,sales_reps,7,"CREATE TABLE sales_reps (\tid integer,\tname b..."
2,table,region,region,222,"CREATE TABLE region (\tid integer,\tname bpchar)"
3,table,orders,orders,223,"CREATE TABLE orders (\tid integer,\taccount_id..."
4,table,accounts,accounts,583,"CREATE TABLE accounts (\tid integer,\tname bpc..."


In [6]:
pd.read_sql_query(sql='''
SELECT *
FROM orders;
''', con=conn)

,id,account_id,occurred_at,standard_qty,gloss_qty,poster_qty,total,standard_amt_usd,gloss_amt_usd,poster_amt_usd,total_amt_usd
0,1,1001,2015-10-06 17:31:14,123,22,24,169,613.77,164.78,194.88,973.43
1,2,1001,2015-11-05 03:34:33,190,41,57,288,948.10,307.09,462.84,1718.03
2,3,1001,2015-12-04 04:21:55,85,47,0,132,424.15,352.03,0.00,776.18
3,4,1001,2016-01-02 01:18:24,144,32,0,176,718.56,239.68,0.00,958.24
4,5,1001,2016-02-01 19:27:27,108,29,28,165,538.92,217.21,227.36,983.49
...,...,...,...,...,...,...,...,...,...,...,...
6907,6908,4501,2016-06-29 04:03:39,11,199,59,269,54.89,1490.51,479.08,2024.48
6908,6909,4501,2016-07-29 19:58:32,5,91,96,192,24.95,681.59,779.52,1486.06
6909,6910,4501,2016-08-27 00:58:11,16,94,82,192,79.84,704.06,665.84,1449.74
6910,6911,4501,2016-11-22 06:52:22,63,67,81,211,314.37,501.83,657.72,1473.92


Task 1:

Which account (by name) placed the earliest order? Your solution should have the account name and the date of the order.

In [7]:
pd.read_sql_query(sql='''
SELECT account_id, name, occurred_at
FROM orders o
JOIN accounts a
ON o.account_id = a.id
ORDER BY occurred_at
LIMIT 1;
''', con=conn)

,account_id,name,occurred_at
0,2861,DISH Network,2013-12-04 04:22:44


Solution:

SELECT a.name, o.occurred_at
FROM accounts a
JOIN orders o
ON a.id = o.account_id
ORDER BY occurred_at
LIMIT 1;

Task 2:

Find the total sales in usd for each account. You should include two columns - the total sales for each company's orders in usd and the company name.

In [36]:
pd.read_sql_query(sql='''
SELECT name, SUM(total) total_sales
FROM orders o
JOIN accounts a
ON o.account_id = a.id
GROUP BY name;
''', con=conn)

,name,total_sales
0,3M,19747
1,ADP,28629
2,AECOM,3549
3,AES,1971
4,AIG,1515
...,...,...
345,World Fuel Services,1719
346,Xcel Energy,3498
347,Xerox,1496
348,Yum Brands,5153


Solution:

SELECT a.name, SUM(total_amt_usd) total_sales
FROM orders o
JOIN accounts a
ON a.id = o.account_id
GROUP BY a.name;

Task 3:

Via what channel did the most recent (latest) web_event occur, which account was associated with this web_event? Your query should return only three values - the date, channel, and account name.

In [11]:
pd.read_sql_query(sql='''
SELECT occurred_at, channel, name
FROM web_events w
JOIN accounts a
ON w.account_id = a.id
ORDER BY occurred_at DESC
LIMIT 1;
''', con=conn)

,occurred_at,channel,name
0,2017-01-01 23:51:09,organic,Molina Healthcare


Solution:

SELECT w.occurred_at, w.channel, a.name
FROM web_events w
JOIN accounts a
ON w.account_id = a.id 
ORDER BY w.occurred_at DESC
LIMIT 1;

Task 4:

Find the total number of times each type of channel from the web_events was used. Your final table should have two columns - the channel and the number of times the channel was used.

In [13]:
pd.read_sql_query(sql='''
SELECT channel, COUNT(*) count
FROM web_events w
GROUP BY channel
ORDER BY channel;
''', con=conn)

,channel,count
0,adwords,906
1,banner,476
2,direct,5298
3,facebook,967
4,organic,952
5,twitter,474


Solution:

SELECT w.channel, COUNT(*)
FROM web_events w
GROUP BY w.channel

Task 5:

Who was the primary contact associated with the earliest web_event?

In [19]:
pd.read_sql_query(sql='''
SELECT occurred_at, channel, name, primary_poc
FROM web_events w
JOIN accounts a
ON w.account_id = a.id
ORDER BY occurred_at
LIMIT 1
''', con=conn)

,occurred_at,channel,name,primary_poc
0,2013-12-04 04:18:29,direct,DISH Network,Leana Hawker


Solution:

SELECT a.primary_poc
FROM web_events w
JOIN accounts a
ON a.id = w.account_id
ORDER BY w.occurred_at
LIMIT 1;

Task 6:

What was the smallest order placed by each account in terms of total usd. Provide only two columns - the account name and the total usd. Order from smallest dollar amounts to largest.

In [40]:
pd.read_sql_query(sql='''
SELECT name, MIN(total_amt_usd) smallest_order
FROM orders o
JOIN accounts a
ON o.account_id = a.id
GROUP BY name
ORDER BY MIN(total_amt_usd);
''', con=conn)
# not necessary ORDER BY smallest_order

,name,smallest_order
0,BlackRock,0.00
1,Boeing,0.00
2,Chevron,0.00
3,Citigroup,0.00
4,Disney,0.00
...,...,...
345,Mohawk Industries,3025.65
346,Waste Management,3101.51
347,General Motors,4393.44
348,Berkshire Hathaway,7474.32


Solution:

SELECT a.name, MIN(total_amt_usd) smallest_order
FROM accounts a
JOIN orders o
ON a.id = o.account_id
GROUP BY a.name
ORDER BY smallest_order;

Task 7:

Find the number of sales reps in each region. Your final table should have two columns - the region and the number of sales_reps. Order from fewest reps to most reps.

In [42]:
pd.read_sql_query(sql='''
SELECT r.name, COUNT(*) num_reps
FROM sales_reps s
JOIN region r
ON s.region_id = r.id
GROUP BY r.name
ORDER BY num_reps;
''', con=conn)

,name,num_reps
0,Midwest,9
1,Southeast,10
2,West,10
3,Northeast,21


Solution:

SELECT r.name, COUNT(*) num_reps
FROM region r
JOIN sales_reps s
ON r.id = s.region_id
GROUP BY r.name
ORDER BY num_reps;